<a href="https://colab.research.google.com/github/DONDAJIN/Kaggle_amex/blob/master/adversarial_val.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 166 kB/s 


In [54]:
import cupy
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from catboost import Pool
from catboost import CatBoost
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
import gc
import warnings
warnings.simplefilter('ignore')

In [34]:
test = pd.read_parquet('/content/drive/MyDrive/Kaggle/Input/test.parquet')
test = test.drop_duplicates(subset=["customer_ID"], keep="last") #subsetで重複している列を指定、keepで最後の行を保つ
test['S_2'] = pd.to_datetime(test['S_2'])
test['month'] = (test['S_2'].dt.month).astype('int8')

train = pd.read_parquet('/content/drive/MyDrive/Kaggle/Input/train.parquet')
train = train.drop_duplicates(subset=["customer_ID"], keep="last")
train['S_2'] = pd.to_datetime(train['S_2'])
train['month'] = (train['S_2'].dt.month).astype('int8')

In [35]:
train['target']=1
test['target']=0

test_private=test[test['month']==4].reset_index(drop=True) #さっきdropした分のindexを整理する
test_public=test[test['month']==10].reset_index(drop=True)
del test
gc.collect()

986

In [46]:
test_public.isnull().sum()

customer_ID       0
S_2               0
P_2            1955
D_39              0
B_1               0
               ... 
D_143             0
D_144             0
D_145             0
month             0
target            0
Length: 192, dtype: int64

Train vs. Public

In [47]:
data=pd.concat([train,test_public]).reset_index(drop=True) #デフォルトでaxis=0

In [48]:
TARGET='target'
drop_cols=['S_2','month','customer_ID',TARGET]
use_cols=[c for c in data.columns if  c not in drop_cols]

In [49]:
cat_params={
        'loss_function' : 'Logloss', #二値分類でtrainとtestの予測を行う
        'eval_metric' : 'AUC',
        'learning_rate': 0.1,
        'num_boost_round': 2500,
        'early_stopping_rounds': 50,
        'random_state': 127,
        'task_type': 'GPU'
}

In [50]:
data

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,month,target
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2018-03-13,0.934745,0,0.009382,1.007647,0.006104,0.135021,0.000000,0.007174,...,-1,0,0,0.000000,NaN,0,0.002970,0,3,1
1,00000fd6641609c6ece5454664794f0340ad84dddce9a2...,2018-03-25,0.880519,6,0.034684,1.004028,0.006911,0.165509,0.000000,0.005068,...,-1,0,0,0.000000,NaN,0,0.003169,0,3,1
2,00001b22f846c82c51f6e3958ccd81970162bae8b007e8...,2018-03-12,0.880875,0,0.004284,0.812649,0.006450,NaN,0.000000,0.007196,...,-1,0,0,0.000000,NaN,0,0.000834,0,3,1
3,000041bdba6ecadd89a52d11886e8eaaec9325906c9723...,2018-03-29,0.621776,0,0.012564,1.006183,0.007829,0.287766,0.000000,0.009937,...,-1,0,0,0.000000,NaN,0,0.005560,0,3,1
4,00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8a...,2018-03-30,0.871900,0,0.007679,0.815746,0.001247,NaN,0.000000,0.005528,...,-1,0,0,0.000000,NaN,0,0.006944,0,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915563,fffed528456683bc080e85131cdcd4bd1a067fcbeca9d4...,2019-10-12,1.003760,1,0.024083,0.814736,0.000000,0.173170,0.000000,0.003942,...,-1,0,0,0.000000,NaN,0,0.007206,0,10,0
915564,fffed768c2776427eb09b7665df629f647228c0654017e...,2019-10-19,0.871654,0,0.178124,0.270553,0.000000,0.138044,0.000000,0.302046,...,-1,0,0,0.000000,NaN,0,0.001633,0,10,0
915565,fffed845116a1a9c7c4b2f0fff839e7aa418e3c0307e8c...,2019-10-08,0.649229,0,0.014754,0.668239,0.000000,0.130658,0.475543,0.010700,...,-1,0,0,0.000000,NaN,0,0.003888,0,10,0
915566,ffff1cf18b40bacfe82d222c6d214eb8400c5a82b207ff...,2019-10-27,-0.002841,0,0.013448,0.002159,0.000000,0.537104,0.654503,0.009408,...,-1,1,0,0.866755,0.020442,1,0.006395,4,10,0


In [51]:
drop_feats=[]
fi_df_all=[]
while True:
  train_x,valid_x,train_y,valid_y=train_test_split(data[use_cols],data[TARGET],test_size=0.33,random_state=42)
  
  trn_data=Pool(train_x,label=train_y)
  val_data=Pool(valid_x,label=valid_y)

  model=CatBoost(cat_params)
  model.fit(trn_data,
            eval_set=val_data,
            verbose_eval=500,
            use_best_model=True)
  pred=model.predict(val_data)
  auc_score=roc_auc_score(valid_y,pred)
  print(f'AUC Score: {auc_score}')

  if auc_score<0.65:
    break
  else:
    fi_df=pd.DataFrame()
    fi_df['importance']=model.get_feature_importance(Pool(train_x,train_y))
    fi_df['features']=use_cols
    fi_df=fi_df.sort_values(by='importance',ascending=False)#降順でsort
    fi_df_all.append(fi_df)
    drop_feats+=fi_df['features'].to_list()[:5] #5個重要度が高いものを落とす
    use_cols=[c for c in use_cols if c not in drop_feats]
    print(f'Drop Features: {drop_feats}')

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9994538	best: 0.9994538 (0)	total: 17.1ms	remaining: 42.8s
500:	test: 0.9999622	best: 0.9999622 (489)	total: 6.79s	remaining: 27.1s
bestTest = 0.9999650717
bestIteration = 838
Shrink model to first 839 iterations.
AUC Score: 0.9999650415639555
Drop Features: ['B_29', 'R_1', 'D_59', 'S_11', 'S_15']


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7515687	best: 0.7515687 (0)	total: 15ms	remaining: 37.6s
500:	test: 0.9322056	best: 0.9322056 (500)	total: 6.79s	remaining: 27.1s
1000:	test: 0.9381778	best: 0.9381778 (1000)	total: 13.5s	remaining: 20.2s
1500:	test: 0.9405165	best: 0.9405165 (1500)	total: 20.2s	remaining: 13.5s
2000:	test: 0.9418837	best: 0.9418837 (2000)	total: 27.1s	remaining: 6.75s
2499:	test: 0.9426577	best: 0.9426577 (2499)	total: 33.7s	remaining: 0us
bestTest = 0.9426577091
bestIteration = 2499
AUC Score: 0.942657786646393
Drop Features: ['B_29', 'R_1', 'D_59', 'S_11', 'S_15', 'S_9', 'S_24', 'D_121', 'S_27', 'S_22']


Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.5605444	best: 0.5605444 (0)	total: 17.4ms	remaining: 43.5s
500:	test: 0.6498894	best: 0.6498894 (500)	total: 6.41s	remaining: 25.6s
1000:	test: 0.6565822	best: 0.6565822 (1000)	total: 12.8s	remaining: 19.2s
1500:	test: 0.6588548	best: 0.6588714 (1495)	total: 19.8s	remaining: 13.1s
bestTest = 0.659254849
bestIteration = 1582
Shrink model to first 1583 iterations.
AUC Score: 0.6592548289333965
Drop Features: ['B_29', 'R_1', 'D_59', 'S_11', 'S_15', 'S_9', 'S_24', 'D_121', 'S_27', 'S_22', 'D_45', 'R_27', 'D_62', 'S_13', 'D_91', 'D_39', 'D_77', 'D_42', 'B_8', 'D_142', 'P_4', 'B_17', 'P_3', 'D_120', 'S_17', 'D_115', 'D_118', 'D_47', 'S_18', 'D_61', 'D_119', 'S_7', 'R_26', 'S_3', 'D_52', 'D_46', 'S_6', 'B_9', 'D_43', 'D_64', 'B_16', 'B_38', 'D_73', 'P_2', 'D_56']


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5583659	best: 0.5583659 (0)	total: 14.6ms	remaining: 36.5s
500:	test: 0.6412795	best: 0.6412795 (500)	total: 6.43s	remaining: 25.7s
1000:	test: 0.6473435	best: 0.6473506 (999)	total: 13s	remaining: 19.4s
1500:	test: 0.6492456	best: 0.6492698 (1487)	total: 19.7s	remaining: 13.1s
2000:	test: 0.6502200	best: 0.6502419 (1989)	total: 26.3s	remaining: 6.55s
bestTest = 0.6503839493
bestIteration = 2126
Shrink model to first 2127 iterations.
AUC Score: 0.6503839674515495
Drop Features: ['B_29', 'R_1', 'D_59', 'S_11', 'S_15', 'S_9', 'S_24', 'D_121', 'S_27', 'S_22', 'D_45', 'R_27', 'D_62', 'S_13', 'D_91', 'D_39', 'D_77', 'D_42', 'B_8', 'D_142', 'P_4', 'B_17', 'P_3', 'D_120', 'S_17', 'D_115', 'D_118', 'D_47', 'S_18', 'D_61', 'D_119', 'S_7', 'R_26', 'S_3', 'D_52', 'D_46', 'S_6', 'B_9', 'D_43', 'D_64', 'B_16', 'B_38', 'D_73', 'P_2', 'D_56', 'B_20', 'B_12', 'D_141', 'D_48', 'B_3']


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5521728	best: 0.5521728 (0)	total: 13.9ms	remaining: 34.7s
500:	test: 0.6326305	best: 0.6326305 (500)	total: 6.15s	remaining: 24.5s
1000:	test: 0.6370685	best: 0.6370685 (1000)	total: 12.3s	remaining: 18.5s
1500:	test: 0.6387317	best: 0.6387627 (1488)	total: 18.6s	remaining: 12.4s
bestTest = 0.6389982402
bestIteration = 1657
Shrink model to first 1658 iterations.
AUC Score: 0.6389982642681281


In [52]:
print(f'Result of Drop Features: {drop_feats}')
print(f'num of Drop Features: {len(drop_feats)}')

Result of Drop Features: ['B_29', 'R_1', 'D_59', 'S_11', 'S_15', 'S_9', 'S_24', 'D_121', 'S_27', 'S_22', 'D_45', 'R_27', 'D_62', 'S_13', 'D_91', 'D_39', 'D_77', 'D_42', 'B_8', 'D_142', 'P_4', 'B_17', 'P_3', 'D_120', 'S_17', 'D_115', 'D_118', 'D_47', 'S_18', 'D_61', 'D_119', 'S_7', 'R_26', 'S_3', 'D_52', 'D_46', 'S_6', 'B_9', 'D_43', 'D_64', 'B_16', 'B_38', 'D_73', 'P_2', 'D_56', 'B_20', 'B_12', 'D_141', 'D_48', 'B_3']
num of Drop Features: 50


In [57]:
fi_df

,importance,features
35,2.460794,B_20
21,2.328765,B_12
139,2.297328,D_141
8,2.289092,D_48
3,2.249872,B_3
...,...,...
92,0.007268,R_24
80,0.005439,R_19
99,0.003902,D_103
76,0.000000,R_18


## Check Featuers

In [ ]:
plt.figure(figsize=(20,15))
for i,feat in enumerate(drop_feats):
  plt.subplot(5,10,i+1)
  sns.displot(train[feat],label='Train')
  sns.displot(test_public[feat],label='Test')
  plt.legend()
  #遅すぎた

In [68]:
public_drop_feats=['B_29', 'R_1', 'D_59', 'S_11', 'S_15', 'S_9', 'S_24', 'D_121', 'S_27', 'S_22', 'D_45', 'R_27', 'D_62', 'S_13', 'D_91', 'D_39', 'D_77', 'D_42', 'B_8', 'D_142', 'P_4', 'B_17', 'P_3', 'D_120', 'S_17', 'D_115', 'D_118', 'D_47', 'S_18', 'D_61', 'D_119', 'S_7', 'R_26', 'S_3', 'D_52', 'D_46', 'S_6', 'B_9', 'D_43', 'D_64', 'B_16', 'B_38', 'D_73', 'P_2', 'D_56', 'B_20', 'B_12', 'D_141', 'D_48', 'B_3']

In [69]:
for feat in public_drop_feats:
  print(f'=========------------{feat}=====================')
  print(f'Train NaN Ratio: {train[feat].isnull().sum()/len(train)}')
  print(f'Public NaN Ratio: {test_public[feat].isnull().sum()/len(test_public)}')

=========------------B_29=====================
Train NaN Ratio: 0.9404593027436573
Public NaN Ratio: 2.6278043599654006e-05
=========------------R_1=====================
Train NaN Ratio: 0.0
Public NaN Ratio: 0.0
=========------------D_59=====================
Train NaN Ratio: 0.0
Public NaN Ratio: 0.0
=========------------S_11=====================
Train NaN Ratio: 0.0
Public NaN Ratio: 0.0
=========------------S_15=====================
Train NaN Ratio: 0.0
Public NaN Ratio: 0.0
=========------------S_9=====================
Train NaN Ratio: 0.40063802943041493
Public NaN Ratio: 0.20101389451555332
=========------------S_24=====================
Train NaN Ratio: 0.003778493962908002
Public NaN Ratio: 0.0025117430007335954
=========------------D_121=====================
Train NaN Ratio: 0.013111417632536015
Public NaN Ratio: 0.01545805914749647
=========------------S_27=====================
Train NaN Ratio: 0.25531200903003404
Public NaN Ratio: 0.21478139952480538
=========------------S_22

In [ ]:
#trainとprivateの場合も同様にして

In [59]:
data=pd.concat([train,test_private]).reset_index(drop=True)

In [60]:
TARGET='target'
drop_cols=['S_2','month','customer_ID',TARGET]
use_cols=[c for c in data.columns if c not in drop_cols]

In [61]:
cat_params = {
        'loss_function' : 'Logloss',
        'eval_metric' : 'AUC',
        'learning_rate': 0.1,
        'num_boost_round': 2500,
        'early_stopping_rounds': 50,
        'random_state': 127,
        'task_type': 'GPU'
    }

In [63]:
drop_feats=[]
fi_df_all=[]
while True:
  train_x,valid_x,train_y,valid_y=train_test_split(data[use_cols],data[TARGET],test_size=0.33,random_state=42)

  trn_data=Pool(train_x,label=train_y)
  val_data=Pool(valid_x,label=valid_y)

  model=CatBoost(cat_params)
  model.fit(trn_data,
            eval_set=val_data,
            verbose_eval=500,
            use_best_model=True
            )
  pred=model.predict(val_data)
  auc_score=roc_auc_score(valid_y,pred)
  print(f'AUC Score: {auc_score}')

  if auc_score<0.65:
    break

  else:
    fi_df=pd.DataFrame()
    fi_df['importance']=model.get_feature_importance(Pool(train_x,train_y))
    fi_df['features']=use_cols
    fi_df=fi_df.sort_values(by='importance',ascending=False)
    fi_df_all.append(fi_df)
    drop_feats+=fi_df['features'].to_list()[:5]
    use_cols=[c for c in use_cols if c not in drop_feats]
    print(f'Drop Features: {drop_feats}')
  
  

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.9976612	best: 0.9976612 (0)	total: 15.8ms	remaining: 39.4s
500:	test: 0.9997217	best: 0.9997217 (500)	total: 6.93s	remaining: 27.7s
1000:	test: 0.9997486	best: 0.9997488 (984)	total: 13.9s	remaining: 20.8s
bestTest = 0.9997587204
bestIteration = 1334
Shrink model to first 1335 iterations.
AUC Score: 0.9997587053769127
Drop Features: ['R_1', 'D_59', 'S_11', 'S_9', 'S_27']


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6639886	best: 0.6639886 (0)	total: 16ms	remaining: 39.9s
500:	test: 0.8684637	best: 0.8684637 (500)	total: 6.79s	remaining: 27.1s
1000:	test: 0.8754429	best: 0.8754429 (1000)	total: 13.6s	remaining: 20.4s
1500:	test: 0.8781965	best: 0.8781965 (1500)	total: 20.6s	remaining: 13.7s
2000:	test: 0.8793626	best: 0.8793626 (2000)	total: 28s	remaining: 6.97s
2499:	test: 0.8802210	best: 0.8802267 (2496)	total: 35s	remaining: 0us
bestTest = 0.8802266717
bestIteration = 2496
Shrink model to first 2497 iterations.
AUC Score: 0.8802266419784869
Drop Features: ['R_1', 'D_59', 'S_11', 'S_9', 'S_27', 'D_121', 'R_27', 'S_15', 'S_24', 'S_22']


Default metric period is 5 because AUC is/are not implemented for GPU


...

0:	test: 0.5499744	best: 0.5499744 (0)	total: 16.4ms	remaining: 41.1s
500:	test: 0.6749360	best: 0.6749360 (500)	total: 6.85s	remaining: 27.3s
1000:	test: 0.6825574	best: 0.6825703 (998)	total: 13.8s	remaining: 20.7s
1500:	test: 0.6853094	best: 0.6853094 (1491)	total: 20.6s	remaining: 13.7s
bestTest = 0.6864766479
bestIteration = 1820
Shrink model to first 1821 iterations.
AUC Score: 0.686476669260057
Drop Features: ['R_1', 'D_59', 'S_11', 'S_9', 'S_27', 'D_121', 'R_27', 'S_15', 'S_24', 'S_22', 'D_39', 'D_62', 'D_45', 'B_17', 'D_60', 'D_77', 'S_17', 'S_13', 'B_40', 'R_26', 'D_91', 'P_3', 'S_18', 'D_120', 'D_115', 'P_4', 'D_61', 'D_118', 'S_3', 'D_119']


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5420743	best: 0.5420743 (0)	total: 14.3ms	remaining: 35.6s
500:	test: 0.6485435	best: 0.6485435 (500)	total: 6.81s	remaining: 27.2s
1000:	test: 0.6544394	best: 0.6544455 (996)	total: 13.5s	remaining: 20.3s
1500:	test: 0.6565936	best: 0.6566077 (1469)	total: 20.4s	remaining: 13.6s
bestTest = 0.6566077471
bestIteration = 1469
Shrink model to first 1470 iterations.
AUC Score: 0.6566077798248706
Drop Features: ['R_1', 'D_59', 'S_11', 'S_9', 'S_27', 'D_121', 'R_27', 'S_15', 'S_24', 'S_22', 'D_39', 'D_62', 'D_45', 'B_17', 'D_60', 'D_77', 'S_17', 'S_13', 'B_40', 'R_26', 'D_91', 'P_3', 'S_18', 'D_120', 'D_115', 'P_4', 'D_61', 'D_118', 'S_3', 'D_119', 'D_42', 'D_73', 'B_8', 'S_7', 'D_47']


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.5366656	best: 0.5366656 (0)	total: 13.4ms	remaining: 33.5s
500:	test: 0.6330339	best: 0.6330553 (499)	total: 6.71s	remaining: 26.8s
1000:	test: 0.6380767	best: 0.6380918 (996)	total: 13.5s	remaining: 20.2s
1500:	test: 0.6395527	best: 0.6395527 (1500)	total: 20.4s	remaining: 13.6s
bestTest = 0.6399492621
bestIteration = 1674
Shrink model to first 1675 iterations.
AUC Score: 0.639949260937448


In [64]:
for feat in drop_feats:
    print(f'========================= {feat} =========================')
    print('Train Nan Ratio:',train[feat].isnull().sum()/len(train))
    print('Privat Nan Ratio:',test_private[feat].isnull().sum()/len(test_private))

========================= R_1 =========================
Train Nan Ratio: 0.0
Privat Nan Ratio: 0.0
========================= D_59 =========================
Train Nan Ratio: 0.0
Privat Nan Ratio: 0.0
========================= S_11 =========================
Train Nan Ratio: 0.0
Privat Nan Ratio: 0.0
========================= S_9 =========================
Train Nan Ratio: 0.40063802943041493
Privat Nan Ratio: 0.2325254398823846
========================= S_27 =========================
Train Nan Ratio: 0.25531200903003404
Privat Nan Ratio: 0.23636118863336225
========================= D_121 =========================
Train Nan Ratio: 0.013111417632536015
Privat Nan Ratio: 0.021281460618933854
========================= R_27 =========================
Train Nan Ratio: 0.06261753317077529
Privat Nan Ratio: 0.00696418115846023
========================= S_15 =========================
Train Nan Ratio: 0.0
Privat Nan Ratio: 0.0
========================= S_24 =========================
Train Nan Ratio

In [66]:
public_drop_feats=['B_29', 'R_1', 'D_59', 'S_11', 'S_15', 'S_9', 'S_24', 'D_121', 'S_27', 'S_22', 'D_45', 'R_27', 'D_62', 'S_13', 'D_91', 'D_39', 'D_77', 'D_42', 'B_8', 'D_142', 'P_4', 'B_17', 'P_3', 'D_120', 'S_17', 'D_115', 'D_118', 'D_47', 'S_18', 'D_61', 'D_119', 'S_7', 'R_26', 'S_3', 'D_52', 'D_46', 'S_6', 'B_9', 'D_43', 'D_64', 'B_16', 'B_38', 'D_73', 'P_2', 'D_56', 'B_20', 'B_12', 'D_141', 'D_48', 'B_3']
private_drop_feats=['R_1', 'D_59', 'S_11', 'S_9', 'S_27', 'D_121', 'R_27', 'S_15', 'S_24', 'S_22', 'D_39', 'D_62', 'D_45', 'B_17', 'D_60', 'D_77', 'S_17', 'S_13', 'B_40', 'R_26', 'D_91', 'P_3', 'S_18', 'D_120', 'D_115', 'P_4', 'D_61', 'D_118', 'S_3', 'D_119', 'D_42', 'D_73', 'B_8', 'S_7', 'D_47']
only_private_drop_feats=[f for f in private_drop_feats if f not in public_drop_feats]
only_private_drop_feats

['D_60', 'B_40']

In [67]:
only_public_feats=[f for f in public_drop_feats if f not in private_drop_feats]
only_public_feats

['B_29',
 'D_142',
 'D_52',
 'D_46',
 'S_6',
 'B_9',
 'D_43',
 'D_64',
 'B_16',
 'B_38',
 'P_2',
 'D_56',
 'B_20',
 'B_12',
 'D_141',
 'D_48',
 'B_3']

# Result
* public_drop_featsだけに入っていてそれが予測の基準になっているものは危ない
* only_private_drop_featsはmodelの特徴量に入れてあげるとよりprivateの予測が上手くできそう
* B_29はnull率も大きくtrainと違うからそれが特徴量に入るとpublicを上手く予測出来てもprivateを上手く予測できるとは限らなくなる